# 2-5 順伝搬関数の実装

## 関数 decode を実装
ここではあとから実装する Detect クラス内で使用する関数 decode を実装する．
decode 関数ではデフォルトボックス$(cx_d, cy_d, w_d, h_d)$とオフセット情報 $loc = (\Delta cx, \Delta cy, \Delta w, \Delta h)$ からバウンディングボックスの座標情報 $(cx, cy, w, h)$を作成する．

## Non-Maximum Supression を行う関数を実装
続いて，同じく Detector クラス内で使用する関数 nm_supression を実装する．
Non−Muximum Supression は画像中の同じ物体に対してフィッティングした僅かに異なる複数個のバウンディングボックスについて，冗長なバウンディングボックスを削除し1つの物体に対しては1つのバウンディングボックスのみを残す処理である．
実装としては，バウンディングボックス同士のかぶっている面積が閾値（ここでは0.45）以上である場合には同じ物体へのバウンディングボックスだとみなし，確信度の最も大きなものを残して他は削除するという操作を行う．

次に示すコードにおいて，scores は確信度が一定以上の値（ここでは0.01）となったデフォルトボックスに対する確信度である．
そのテンソルサイズは確信度閾値を超えたデフォルトボックスの数と同じになる．

## クラス Detect を実装
SSD の推論時には Detect クラスを適用して $(batch\_num, 21, 200, 5)$ のテンソルを出力する．
テンソルの先頭はミニバッチの番号，2番目が各クラスのインデックスを示す次元，3番目が信頼度上位200個のうち何番目のバウンディングボックスか，最後がバウンディングボックスの情報 $(conf, x_{min}, y_{min}, x_{max}, y_{max})$ を表している．
Detect クラスには次の3つの要素を入力する．
- loc モジュールの出力 $(batch\_num, 8732, 4)$
- conf モジュールの出力 $(batch\_num, 8732, 21)$
- デフォルトボックスの情報 $(8732, 4)$

このうち，conf モジュールの出力はソフトマックス関数で規格化しておく．
実装では torch.autograd.Function クラスを継承している．
処理の流れは以下の通り．
1. 関数 decode を使ってデフォルトボックスとオフセット情報からバウンディングボックスに変換
1. 確信度が閾値以上のバウンディングボックスの取り出し
1. nm_supression 関数を適用しかぶっているバウンディングボックスを削除

## SSD モデルを実装
最後に SSD モデルの順伝播を実装する．
先にも述べたとおり，vgg，extras モジュールから source1~6 を取り出し，オフセット情報 loc と確信度 conf を取り出す．
最終的な出力として，変数 output にオフセット情報と確信度，デフォルトボックスの情報をまとめる．
学習時には，この変数 output が出力となる．
推論時には Detect クラスの順伝播関数を使ってバウンディングボックスの情報を出力する．